In [121]:
import pandas as pd

## in the below calculation we assume that for every valid model signal, we buy 1 contract at a cost of current_kalshi_price

df = pd.read_csv('kalshi-historical-3-30-25-with-y.csv', usecols=["time","contract_type","strike_price","pdf_estimate","current_kalshi_price","outcome"]) # this ALWAYS returns a DataFrame
df['time'] = pd.to_datetime(df['time'])
df =  df[df['time'] > pd.Timestamp('2025-03-23')]
df.head()


,time,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
77320,2025-03-24 11:27:28,No,89250,70.92,90,No
77321,2025-03-24 11:27:28,Yes,89250,29.08,16,No
77322,2025-03-24 11:27:28,No,88750,58.15,79,No
77323,2025-03-24 11:27:28,Yes,88750,41.85,28,No
77324,2025-03-24 11:27:28,No,88250,45.98,62,No


In [120]:
# our model ONLY makes trades on 10c price differences, with a floor probability of 50, and only before 5pm on the current day. This is a boolean series
cond = (
    (df["pdf_estimate"] >= df["current_kalshi_price"] + 10)
    & (df["current_kalshi_price"] >= 50)
    & (df["time"].dt.hour < 17)
)

filtered_df = df[cond].copy() # This is applying the boolean series to df and making a copy of the result and putting it into filtered_df

totalRows = len(df) # ~40,000 rows
trades = len(filtered_df)
print(f"total trades:  {trades}/{totalRows}") # accurate because we use original df AND the cond

filtered_df["payout"] = 0.0 # add a new column to the dataframe, and set all rows to 0, yay for vectorization!
average_entry_price = filtered_df["current_kalshi_price"].mean()

filtered_df["payout"] = -filtered_df["current_kalshi_price"] / 100  # assume lose by default

hits = filtered_df["outcome"] == filtered_df["contract_type"] # boolean series
filtered_df.loc[hits, "payout"] = (100 - filtered_df.loc[hits, "current_kalshi_price"]) / 100 # ($1 - contract_cost) / 100 (profit)

# now let's handle the misses. We scale this to dollars
filtered_df.loc[~hits, "payout"] = -filtered_df.loc[~hits, "current_kalshi_price"] / 100 # payout here is -contract_cost (lost our whole contract value)

# now we sum the results
total_profit = filtered_df["payout"].sum()

wins = (filtered_df['outcome'] == filtered_df['contract_type']).sum()
print(f"number of winners: {wins}")
print(f"win rate: {wins/trades}")
print(f"mean contract entry price:  {average_entry_price:.2f}")
print(f"profit: {total_profit}")
# let's compute average entry price of ALL trades (pray it's around 60c?)

filtered_df.head()

total trades:  647/39812
number of winners: 393
win rate: 0.60741885625966
mean contract entry price:  59.88
profit: 5.599999999999994


,time,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome,payout
77825,2025-03-24 13:18:04,Yes,88250,61.31,50,No,-0.50
77833,2025-03-24 13:20:05,Yes,88250,61.00,51,No,-0.51
77925,2025-03-24 13:44:05,Yes,88250,64.77,54,No,-0.54
77931,2025-03-24 13:46:05,Yes,88250,62.42,50,No,-0.50
78061,2025-03-24 14:24:05,Yes,88250,64.16,51,No,-0.51


In [150]:
grouped = filtered_df.groupby([ # we simply groupby(['strike_price'], filtered_df["time"].dt.date) as in price, day
    "strike_price",
    filtered_df["time"].dt.date
])

# contract type doesn't make sense to aggregate on here, so we don't include that column
grouped_sum = grouped.agg({
    "payout": "sum", # definitely want to see how much we win or lose day to day
    "current_kalshi_price": "mean",  # we average out the entry price for that day
    "outcome": "count"  # we choose outcome to show how many trades we made on that day for that strike
}).reset_index()

grouped_sum = grouped_sum.sort_values(by='payout', key=abs, ascending=False)

grouped_sum.head(20)

,strike_price,time,payout,current_kalshi_price,outcome
29,86750,2025-03-27,61.45,64.064327,171
4,82750,2025-03-30,-60.41,61.642857,98
32,87250,2025-03-27,51.08,55.192982,114
22,85750,2025-03-28,-40.18,59.970149,67
7,83750,2025-03-28,15.57,52.818182,33
21,85750,2025-03-27,9.80,66.206897,29
30,86750,2025-03-28,-8.53,53.312500,16
3,82500,2025-03-29,-8.27,51.687500,16
9,84250,2025-03-28,-6.75,51.923077,13
35,88250,2025-03-24,-5.63,51.181818,11


In [160]:
# next let's group by strike price, day, AND contract type. Let's also try renaming outcome to trades by df = df.rename(columns={'col1': 'new_col1', 'col2': 'new_col2'})
anotherGrouping = filtered_df.groupby(['strike_price', filtered_df['time'].dt.date, 'contract_type'])
fullTable = anotherGrouping.agg({
    "payout":"sum",
    "current_kalshi_price":"mean",
    "outcome":"count"
}).rename(columns={'outcome':'numTradesInCategory','current_kalshi_price':'average_entry'}).reset_index()

# of course we should sort by time if we're going to be looking at the table itself


# verify we didn't somehow lose any trades
print(f"total number of trades in this dataset: {fullTable['numTradesInCategory'].sum()}")
fullTable.head()


total number of trades in this dataset: 647


,strike_price,time,contract_type,payout,average_entry,numTradesInCategory
0,81250,2025-03-30,Yes,0.16,84.000000,1
1,81750,2025-03-30,Yes,0.24,76.000000,1
2,82250,2025-03-30,Yes,4.38,56.200000,10
3,82500,2025-03-29,Yes,-8.27,51.687500,16
4,82750,2025-03-30,Yes,-60.41,61.642857,98


In [148]:
df = pd.read_csv("kalshi-historical-3-16-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])

_, _, _, res_df = calculate_total_payout_three_daily_limit_with_limit_sells(df)
res_df

FileNotFoundError: [Errno 2] No such file or directory: 'kalshi-historical-3-16-25-with-y.csv'

In [108]:
import pandas as pd

def calculate_total_payout_limits(df, unit_size=1):
    total_payout = 0
    num_hit = 0
    num_miss = 0

    filtered_rows = []

    # Track active trades for each day (max 3 trades per day)
    trades_per_day = {}

    # Store active trades (contract_key: (entry_time, entry_price))
    active_trades = {}

    # Iterate over the rows in the dataframe
    for idx, row in df.iterrows():
        # Get the current day (we only care about the date part)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"], trade_day)
        
        # Ensure we have a maximum of 3 trades per day
        if trade_day not in trades_per_day:
            trades_per_day[trade_day] = []

        # Entry condition: If we haven't reached 3 trades for the day and the entry condition is met
        if True:
            if row["pdf_estimate"] >= row["current_kalshi_price"] + 10 and row["current_kalshi_price"] >= 50:
                # Register the trade for the day
                trades_per_day[trade_day].append(contract_key)

                # Store the trade entry details
                active_trades[contract_key] = {
                    "entry_time": row["time"],
                    "entry_price": row["current_kalshi_price"],
                    "limit_price": row["pdf_estimate"],
                    "exited": False,
                    "exit_time": None,
                    "exit_price": None,
                    "outcome": row["outcome"]
                }
                # Add this row to the filtered rows (this is the trade entry)
                filtered_rows.append(row)

        # Now we need to check if a limit sell opportunity exists for each active trade
        for trade_contract_key, trade in list(active_trades.items()):
            # Look ahead for a limit sell opportunity (price >= entry_price - 5)
            if not trade["exited"]:
                if trade_day == trade["entry_time"].date() and row["time"] > trade["entry_time"] and trade_contract_key == contract_key and row["current_kalshi_price"] >= trade["limit_price"]:
                    # Mark the trade as exited
                    trade["exit_time"] = row["time"]
                    trade["exit_price"] = trade["limit_price"]
                    trade["exited"] = True

                    # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                    # print(row["time"],row["current_kalshi_price"], contract_key)


                    # Calculate profit (gain or loss)
                    gain = (trade["exit_price"] / trade["entry_price"]) - 1
                    total_payout += gain * unit_size
                    num_hit += 1

    # Handle any remaining active trades at the end of the last day in the dataset
    for trade_contract_key, trade in list(active_trades.items()):
        if not trade["exited"]:
            if trade_contract_key[0] != trade["outcome"]:
                total_payout -= 1  # Apply -1 penalty for missed exit
                num_miss += 1
                trade["exited"] = True
                print(trade_contract_key, trade)
            else:
                total_payout += (trade["limit_price"] / trade["entry_price"]) - 1
                num_hit += 1
                trade["exited"] = True

    # Convert the filtered rows into a DataFrame for results
    res_df = pd.DataFrame(filtered_rows)

    # Print the final summary
    print(f"Net Profit in Units: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")

    return total_payout, num_hit, num_hit + num_miss, res_df



In [109]:
df = pd.read_csv("kalshi-historical-3-30-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])
_, _, _, res_df = calculate_total_payout_limits(df)
res_df

('No', 105000, datetime.date(2025, 1, 21)) {'entry_time': Timestamp('2025-01-21 11:42:00'), 'entry_price': 52, 'limit_price': 62.39, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'Yes'}
('Yes', 103750, datetime.date(2025, 1, 23)) {'entry_time': Timestamp('2025-01-23 15:52:00'), 'entry_price': 52, 'limit_price': 64.07, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 105750, datetime.date(2025, 1, 24)) {'entry_time': Timestamp('2025-01-24 14:46:00'), 'entry_price': 52, 'limit_price': 66.50999999999999, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 100750, datetime.date(2025, 1, 28)) {'entry_time': Timestamp('2025-01-28 16:46:00'), 'entry_price': 59, 'limit_price': 73.57, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No'}
('Yes', 105500, datetime.date(2025, 1, 30)) {'entry_time': Timestamp('2025-01-30 15:36:00'), 'entry_price': 55, 'limit_price': 71.27, 'exited': True, 'exit_time': N

,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
271,2025-01-16 09:44:00,BTC,Daily,Yes,98250,65.01,52,Yes
273,2025-01-16 09:44:00,BTC,Daily,Yes,97750,80.68,64,Yes
289,2025-01-16 09:46:00,BTC,Daily,Yes,97750,78.65,54,Yes
291,2025-01-16 09:46:00,BTC,Daily,Yes,97250,78.65,66,Yes
293,2025-01-16 09:46:00,BTC,Daily,Yes,96750,89.21,76,Yes
...,...,...,...,...,...,...,...,...
116559,2025-03-30 13:02:04,BTC,Daily,Yes,82250,65.77,53,Yes
116569,2025-03-30 13:04:04,BTC,Daily,Yes,82250,65.37,53,Yes
116619,2025-03-30 13:14:05,BTC,Daily,Yes,82250,66.44,56,Yes
116757,2025-03-30 13:46:04,BTC,Daily,Yes,82250,67.01,55,Yes


In [105]:
import pandas as pd
import numpy as np
from datetime import time
from collections import defaultdict


def calculate_total_payout_limits_and_price_history(df, unit_size=1):
    total_payout = 0
    num_hit = 0
    num_miss = 0

    filtered_rows = []

    # Track active trades for each day (max 3 trades per day)
    trades_per_day = {}

    # Store active trades (contract_key: (entry_time, entry_price))
    active_trades = defaultdict(list)

    price_history = {}
    stop_loss_cutoff = time(15, 30)  # "15:30" -> time(15, 30)


    # Iterate over the rows in the dataframe
    for idx, row in df.iterrows():
        # Get the current day (we only care about the date part)
        trade_day = row["time"].date()
        contract_key = (row["contract_type"], row["strike_price"], trade_day)
        # Initialize price history for the asset if not already tracked
        if contract_key not in price_history:
            price_history[contract_key] = []

        # Add current price to the price history for the asset
        price_history[contract_key].append(row["current_kalshi_price"])

        # Calculate the volatility based on the last 10 prices (adjust window size if needed)
        if len(price_history[contract_key]) > 10:
            volatility = np.std(price_history[contract_key][-10:])
        else:
            volatility = 0  # Not enough data for volatility calculation
            
        # Ensure we have a maximum of 3 trades per day
        if trade_day not in trades_per_day:
            trades_per_day[trade_day] = []

        # Entry condition: If we haven't reached 3 trades for the day and the entry condition is met
        if len(trades_per_day[trade_day]) < 99 and row["time"].time() <= stop_loss_cutoff:
            if row["pdf_estimate"] >= row["current_kalshi_price"] + 10 and row["current_kalshi_price"] >= 50:
                # Register the trade for the day
                trades_per_day[trade_day].append(contract_key)

                # Store the trade entry details
                active_trades[contract_key].append({
                    "entry_time": row["time"],
                    "entry_price": row["current_kalshi_price"],
                    "limit_price": row["pdf_estimate"],
                    "exited": False,
                    "exit_time": None,
                    "exit_price": None,
                    "outcome": row["outcome"],
                    "trailing_stop": row["current_kalshi_price"] - volatility * 3
                })
                # Add this row to the filtered rows (this is the trade entry)
                filtered_rows.append(row)

        # Now we need to check if a limit sell opportunity exists for each active trade
        for trade_contract_key, trades in list(active_trades.items()):
            for trade in trades:
                # Look ahead for a limit sell opportunity (price >= entry_price - 5)
                if not trade["exited"]:
                    if trade_day == trade["entry_time"].date() and row["time"] > trade["entry_time"] and trade_contract_key == contract_key:
                        if row["current_kalshi_price"] >= trade["limit_price"]:

                            # Mark the trade as exited
                            trade["exit_time"] = row["time"]
                            trade["exit_price"] = trade["limit_price"]
                            trade["exited"] = True

                            # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                            # print(row["time"],row["current_kalshi_price"], contract_key)

                            # Calculate profit (gain or loss)
                            gain = (trade["exit_price"] / trade["entry_price"]) - 1
                            total_payout += gain * unit_size
                            num_hit += 1
                        if row["current_kalshi_price"] <= trade["trailing_stop"] and row["time"].time() >= stop_loss_cutoff:
                            # Mark the trade as exited
                            trade["exit_time"] = row["time"]
                            trade["exit_price"] = row["current_kalshi_price"]
                            trade["exited"] = True

                            # print(trade["entry_time"],trade["entry_price"], trade_contract_key)
                            # print(row["time"],row["current_kalshi_price"], contract_key)

                            # Calculate profit (gain or loss)
                            gain = (trade["entry_price"] - row["current_kalshi_price"])/trade["entry_price"]
                            # print(trade_contract_key, trade)
                            total_payout -= gain * unit_size
                            num_miss += 1

    # Handle any remaining active trades at the end of the last day in the dataset
    for trade_contract_key, trades in list(active_trades.items()):
        for trade in trades:
            if not trade["exited"]:
                if trade_contract_key[0] != trade["outcome"]:
                    total_payout -= 1  # Apply -1 penalty for missed exit
                    num_miss += 1
                    trade["exited"] = True
                    print(trade_contract_key, trade)
                else:
                    total_payout += (trade["limit_price"] / trade["entry_price"]) - 1
                    num_hit += 1
                    trade["exited"] = True

    # Convert the filtered rows into a DataFrame for results
    res_df = pd.DataFrame(filtered_rows)

    # Print the final summary
    print(f"Net Profit in Units: {total_payout}")
    print(f"Total Trades: {num_hit}/{num_hit + num_miss}")
    print(f"Winning Trades: {num_hit}")
    print(f"Losing Trades: {num_miss}")
    return total_payout, num_hit, num_hit + num_miss, res_df



In [106]:
df = pd.read_csv("kalshi-historical-3-30-25-with-y.csv")
df['time'] = pd.to_datetime(df['time'])
cutoff_date = "2025-03-23"
df = df[df["time"] > cutoff_date]
_, _, _, res_df = calculate_total_payout_limits_and_price_history(df)
res_df

('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:38:04'), 'entry_price': 53, 'limit_price': 67.38, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(42.955598574330075)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:40:04'), 'entry_price': 52, 'limit_price': 67.44, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(42.0501256289338)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:42:06'), 'entry_price': 51, 'limit_price': 65.33, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(41.782625102557674)}
('Yes', 87250, datetime.date(2025, 3, 26)) {'entry_time': Timestamp('2025-03-26 20:44:05'), 'entry_price': 51, 'limit_price': 63.57, 'exited': True, 'exit_time': None, 'exit_price': None, 'outcome': 'No', 'trailing_stop': np.float64(43.6454

,time,currency,event_type,contract_type,strike_price,pdf_estimate,current_kalshi_price,outcome
77825,2025-03-24 13:18:04,BTC,Daily,Yes,88250,61.31,50,No
77833,2025-03-24 13:20:05,BTC,Daily,Yes,88250,61.00,51,No
77925,2025-03-24 13:44:05,BTC,Daily,Yes,88250,64.77,54,No
77931,2025-03-24 13:46:05,BTC,Daily,Yes,88250,62.42,50,No
78061,2025-03-24 14:24:05,BTC,Daily,Yes,88250,64.16,51,No
...,...,...,...,...,...,...,...,...
116559,2025-03-30 13:02:04,BTC,Daily,Yes,82250,65.77,53,Yes
116569,2025-03-30 13:04:04,BTC,Daily,Yes,82250,65.37,53,Yes
116619,2025-03-30 13:14:05,BTC,Daily,Yes,82250,66.44,56,Yes
116757,2025-03-30 13:46:04,BTC,Daily,Yes,82250,67.01,55,Yes


In [107]:
daily_counts = res_df.groupby(res_df["time"].dt.date).size()
daily_counts

time
2025-03-24     953
2025-03-25     542
2025-03-26     537
2025-03-27    1275
2025-03-28     160
2025-03-29    1026
2025-03-30     115
dtype: int64

In [100]:
# dates_to_drop = ['2025-02-04', '2025-02-05']

# # Filter out rows with the specified dates
# df = df[~df['time'].dt.date.isin(pd.to_datetime(dates_to_drop).date)]

In [ ]:
# df.to_csv('kalshi-historical-2-4-25-with-y-REMOVED-STALE_DAYS.csv', index=False)